Where to find forms: https://docs.google.com/spreadsheets/d/1-L8rIMTMQGPrmdFsa70k85RWJaZCurad_zug7qFHS68/edit#gid=130175332

# Load Libraries

In [21]:
import urllib.request, json 
from urllib.request import urlopen
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import date
import os
from os import walk
import os.path
from os import path
import uuid

import re
import random

import numpy as np

import hashlib

import requests

from urllib.request import Request, urlopen
from urllib.parse import quote

#!pip install PyPDF2
import PyPDF2

from io import StringIO # https://stackoverflow.com/a/18284900
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

from contextlib import contextmanager
import threading
import _thread

class TimeoutException(Exception): pass
@contextmanager
def time_limit(seconds):
    timer = threading.Timer(seconds, lambda: _thread.interrupt_main())
    timer.start()
    try:
        yield
    except KeyboardInterrupt:
        raise TimeoutException("Timed out.")
    finally:
        # if the action ends in specified time, timer is canceled
        timer.cancel()


def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb') # python three change
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text
   
def download_pdf(url,path,filename):
    
    global csv_path
    
    if not os.path.exists(path):
        os.makedirs(path)
    
    try:
        filename = path+filename

        f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

        with open(filename, "wb") as code:
            #code.write(f.read())
            code.write(f.content)

        return convert(filename)
    except:
        return "error"
    
def hashme(w):
    h = hashlib.md5(w.encode('utf-8'))
    return h.hexdigest()

def get_soup(url):
    if re.search("^file:\/\/",url):
        try:
            with open(re.findall("file:\/\/(.*)",url)[0], 'r') as file:
                page = file.read()
            soup = BeautifulSoup(page)
            return soup
        except:
            print("Error reading file:",re.findall("file:\/\/(.*)",url)[0])
    else:
        try:
            try:
                with time_limit(10):
                    with urllib.request.urlopen(urllib.request.Request(url, headers={'User-agent': 'Mozilla/5.0'}), timeout=15) as url:
                        try:
                            content = url.read().decode()
                        except:
                            content = url.read().decode("iso-8859-1")

                    soup = BeautifulSoup(content)
                    return soup
            except TimeoutException as e:
                print("Timed out!")
                return None
        except:
            print("Error Reading Page!!!")
            return None
    
def pars_link(this_url,href):
    
    if (this_url.split("/")[-1] == "") and (re.search("\.|\?",this_url.split("/")[-1])):
        this_url = this_url.replace(this_url.split("/")[-1],"")

    if this_url[-1]!="/":
        this_url += "/"

    http = re.findall("(https?://)([^/]*)",this_url)[0][0]
    domain = re.findall("(https?://)([^/]*)",this_url)[0][1] 

    if (len(href)>0) & (re.search("^http",href)==None):
        if href[0]=="/":
            output = http + domain + href
        elif href[0]=="#":
            output = "error"
        elif re.search("^javascript",href):
            output = "error"
        else:
            output = this_url + href
    else:
        output = href
            
    return output

def get_links(jurisdiction,url,regex_form,regex_link,goto_depth=0,depth=0):
    
    global min_wait, max_wait, visited_df
    
    print("\nChecking",url)
    soup = get_soup(url)
    
    if soup!=None:

        files_df = pd.DataFrame([],columns=["jurisdiction","source","group","title","url","filename","downloaded"])
        links_df = pd.DataFrame()

        if re.search("^file:\/\/",url):
            source = re.findall("file:\/\/\.\.\/data\/state_forms\/_saved_html\/(.*)\.html?$",url)[0]
            source = re.sub("\\\\","/",source)
            url = "https://" + source
            #print(url)
        else:
            source = re.findall("(https?://)([^/]*)",url)[0][1]

        links = []

        for a in soup.find_all('a', href=True):

            for reg in regex_form:
                href = a['href']    
                href = pars_link(url,href)
                if re.findall(reg,href,re.IGNORECASE):
                    
                    #try:
                    file_id = hashme(href)
                    group = np.nan
                    if len(a.contents)>0:
                        title = a.contents[0].getText()
                    else:
                        title = "UNKNOWN"
                    form_url = href
                    filename = re.findall(reg,href,re.IGNORECASE)[0]
                    downloaded = 0

                    files_df = pd.concat([files_df, pd.DataFrame([[jurisdiction,source,group,title,form_url,filename,downloaded]],columns=["jurisdiction","source","group","title","url","filename","downloaded"],index=[file_id])])            
                    #except:
                    #    print("Error parsing link to form!")

        files_df = files_df[~files_df.index.duplicated(keep='first')]
        print(len(files_df),"possible forms")
        
        for a in soup.find_all('a', href=True):
            
            for reg in regex_link:
                href = a['href']    
                href = pars_link(url,href)
                if (href!="error") and (re.findall(reg,href,re.IGNORECASE)):
                    if ((href not in files_df["url"].values) & (goto_depth > depth)):
                        links_df = pd.concat([links_df, pd.DataFrame([[href,0]],columns=["url","visited"])])

        links_df = links_df.drop_duplicates()
        print(len(links_df),"links to follow")

        if (len(links_df)>0):
            for link in links_df["url"]:
                if (link not in visited_df["link"].values):
                    print("trying link...")
                    time.sleep(random.uniform(min_wait, max_wait))
                    files_df = pd.concat([files_df,get_links(jurisdiction,link,regex_form,regex_link,goto_depth=goto_depth,depth=depth+1)])
                    visited_df = pd.concat([visited_df,pd.DataFrame([[link]],columns=["link"])])
                
        files_df = files_df[~files_df.index.duplicated(keep='first')]
        print(len(files_df),"possible forms this run\n")                
        return files_df
    
    else:
        print("Error reading",url)
        return pd.DataFrame()

min_wait = 0
max_wait = 1

csv_path = "../data/state_forms/"

In [2]:
try:
    forms_df = pd.read_csv(csv_path+"form_data.csv", index_col=0, encoding="utf-8")
except: 
    forms_df = pd.DataFrame([],columns=["id","jurisdiction","source","group","title","url","filename","downloaded"])
    forms_df.set_index(["id"], inplace = True, drop = True)
    
print("Downloaded:",len(forms_df[forms_df["downloaded"]==1]))
print("Pending:",len(forms_df[forms_df["downloaded"]==0]))
print("All Entries:",len(forms_df))

forms_df.head()

Downloaded: 0
Pending: 9668
All Entries: 9668


,jurisdiction,source,group,title,url,filename,downloaded
ce7ec7991f0283ea319598ff05183e50,MA,www.mass.gov,NaN,\n,https://www.mass.gov/doc/order-to-render-mpc-7...,/download,0
13c091b90b3ee286afef69e55591d6f9,MA,www.mass.gov,NaN,\n,https://www.mass.gov/doc/trust-account-mpc-859...,/download,0
c9dfc28730a9379f3cb8959d3be88386,MA,www.mass.gov,NaN,\n,https://www.mass.gov/doc/inventory-without-sch...,/download,0
b154cdf73df83f0155f5978f748ad327,MA,www.mass.gov,NaN,\n,https://www.mass.gov/doc/petition-to-modify-bo...,/download,0
88209861aa5328efab2c0082da6ed5a1,MA,www.mass.gov,NaN,\n,https://www.mass.gov/doc/instructions-for-comp...,/download,0


## Scrape Sites

In [22]:
states = [
    
     {"regex_form":["/download$","([^\/]*\.pdf)"],"regex_link":["\/lists\/"],"goto_depth":1,"jurisdiction":"MA","url":"https://www.mass.gov/guides/court-forms-by-topic"}
    ,{"regex_form":["strFormNumber=(.*$)","([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"ME","url":"https://mjbportal.courts.maine.gov/CourtForms/FormsLists/Index"}    
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"UT","url":"https://legacy.utcourts.gov/forms/"}   
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["forms\/"],"goto_depth":1,"jurisdiction":"IL","url":"https://www.illinoiscourts.gov/documents-and-forms/approved-forms/"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["GetForms\.aspx"],"goto_depth":2,"jurisdiction":"MN","url":"https://www.mncourts.gov/getforms.aspx"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/forms\/","\/court_rules\/"],"goto_depth":1,"jurisdiction":"WA","url":"https://www.courts.wa.gov/forms/"}    
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":1,"jurisdiction":"AL","url":"https://eforms.alacourt.gov"}    
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":1,"jurisdiction":"CT","url":"https://www.jud.ct.gov/webforms/"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"ID","url":"https://courtselfhelp.idaho.gov/Forms"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/legal-forms\/"],"goto_depth":3,"jurisdiction":"KS","url":"https://www.kansasjudicialcouncil.org/legal-forms"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/forms\/"],"goto_depth":1,"jurisdiction":"NV","url":"https://selfhelp.nvcourts.gov/forms?enter=1"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"AK","url":"https://courts.alaska.gov/forms/index.htm"}
    ,{"regex_form":["\?Form=(.*)"],"regex_link":["\/Forms\/"],"goto_depth":2,"jurisdiction":"CO","url":"https://www.courts.state.co.us/Forms/Index.cfm"}
    
    # there are a bunch of zip files here?????
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"GA","url":"https://georgiacourts.gov/statecourt/state-court-behind-the-bench/state-court-forms/"}
    
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/form"],"goto_depth":1,"jurisdiction":"HI","url":"https://www.courts.state.hi.us/self-help/courts/forms/court_forms"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"AR","url":"https://www.arcourts.gov/forms-and-publications/court-forms"}

    # ------ 
    
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"FL","url":"https://help.flcourts.org/Get-Started/Find-a-Form"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"FL","url":"https://www.flcourts.org/Resources-Services/Court-Improvement/Family-Courts/Family-Law-Forms"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"FL","url":"https://www.leeclerk.org/courts/forms-library"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"IA","url":"https://www.iowacourts.gov/for-the-public/court-forms/"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"IN","url":"https://www.in.gov/courts/selfservice/"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"KY","url":"https://kycourts.gov/Legal-Forms/Pages/default.aspx"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"LA","url":"https://www.lsba.org/Public/FindLegalHelp/SelfRepresentation.aspx"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"MD","url":"https://www.courts.state.md.us/courtforms?forms%5B0%5D=languages%3A59"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"MO","url":"https://www.courts.mo.gov/page.jsp?id=103116"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"MS","url":"https://courts.ms.gov/mec/Mec-forms.php"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"MT","url":"https://courts.mt.gov/forms/"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"ND","url":"https://www.ndcourts.gov/legal-self-help"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"ND","url":"https://www.ndcourts.gov/legal-self-help/general-use-forms"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"NH","url":"https://www.courts.nh.gov/our-courts/circuit-court/district-division/forms"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"NJ","url":"https://www.njcourts.gov/selfhelp/catalog.html"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"NM","url":"https://www.nmcourts.gov/self-help/forms/"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"NY","url":"https://www.nycourts.gov/forms/"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"OH","url":"https://www.supremecourt.ohio.gov/JCS/courtSvcs/forms.asp"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"OK","url":"https://www.oscn.net/static/forms/start.asp"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"OR","url":"https://www.courts.oregon.gov/forms/Pages/default.aspx"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"PA","url":"https://www.pacourts.us/forms"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"RI","url":"https://www.courts.ri.gov/PublicResources/forms/Pages/default.aspx"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"SD","url":"https://ujslawhelp.sd.gov/onlineforms.aspx"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"TN","url":"https://www.tncourts.gov/node/707185"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"TX","url":"https://www.txcourts.gov/rules-forms/forms/"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"TX","url":"https://selfhelp.efiletexas.gov/SRL"}
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"WV","url":"http://www.courtswv.gov/legal-community/court-forms.html"}

    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"AZ","url":"https://www.azcourts.gov/selfservicecenter/forms"}
    
    # links not in html
    #,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"MI","url":"https://www.courts.michigan.gov/SCAO-forms/"}
    
    
    # Pull from local HTML because the page is dynamicly rendered
    # - Prepend `file://` to the "url" with what follows being the relative address of an html file
    # - When naming the file, replace any `/` in the url with `\`
    # - To keep track of pages with different content, but the same URL, you can append a `\?p=1` et al 
    
    ,{"regex_form":["Download\.aspx\?id=(.*)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"DE","url":"file://../data/state_forms/_saved_html/courts.delaware.gov\\forms\\?p=1.html"}    
    ,{"regex_form":["Download\.aspx\?id=(.*)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"DE","url":"file://../data/state_forms/_saved_html/courts.delaware.gov\\forms\\?p=2.html"}    
    ,{"regex_form":["Download\.aspx\?id=(.*)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"DE","url":"file://../data/state_forms/_saved_html/courts.delaware.gov\\forms\\?p=3.html"}    
    ,{"regex_form":["Download\.aspx\?id=(.*)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"DE","url":"file://../data/state_forms/_saved_html/courts.delaware.gov\\forms\\?p=4.html"}    
    ,{"regex_form":["Download\.aspx\?id=(.*)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"DE","url":"file://../data/state_forms/_saved_html/courts.delaware.gov\\forms\\?p=5.html"}    
    ,{"regex_form":["Download\.aspx\?id=(.*)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"DE","url":"file://../data/state_forms/_saved_html/courts.delaware.gov\\forms\\?p=6.html"}    
    ,{"regex_form":["Download\.aspx\?id=(.*)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"DE","url":"file://../data/state_forms/_saved_html/courts.delaware.gov\\forms\\?p=7.html"}    
    ,{"regex_form":["Download\.aspx\?id=(.*)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"DE","url":"file://../data/state_forms/_saved_html/courts.delaware.gov\\forms\\?p=8.html"}    
    
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"SC","url":"file://../data/state_forms/_saved_html/www.sccourts.org\\forms\\searchType.cfm.html"}
    
    # page dynamicly loads data
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"DC","url":"file://../data/state_forms/_saved_html/www.dccourts.gov\\services\\forms.html"}
    
    # had to open and re-save html files to utf-8
    ,{"regex_form":["jirc_complaint_form\.pdf","selfev2\.pdf"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"VA","url":"file://../data/state_forms/_saved_html/www.vacourts.gov\\forms\\home.html.html"}                
    ,{"regex_form":[".*form.*\.pdf"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"VA","url":"file://../data/state_forms/_saved_html/www.vacourts.gov\\courtadmin\\aoc\\djs\\programs\\jsc\\forms\\home.html.html"}                
    ,{"regex_form":[".*form.*\.pdf"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"VA","url":"file://../data/state_forms/_saved_html/www.vacourts.gov\\courts\\cav\\forms.html.html"}                
    ,{"regex_form":[".*form.*\.pdf"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"VA","url":"file://../data/state_forms/_saved_html/www.vacourts.gov\\courts\\scv\\forms.html.html"}                
    ,{"regex_form":[".*form.*\.pdf"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"VA","url":"file://../data/state_forms/_saved_html/www.vacourts.gov\\forms\\circuit\\home.html.html"}                
    ,{"regex_form":[".*form.*\.pdf"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"VA","url":"file://../data/state_forms/_saved_html/www.vacourts.gov\\forms\\district\\home.html.html"}                
    ,{"regex_form":[".*form.*\.pdf"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"VA","url":"file://../data/state_forms/_saved_html/www.vacourts.gov\\forms\\district\\jdr.html.html"}                
    ,{"regex_form":[".*form.*\.pdf"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"VA","url":"file://../data/state_forms/_saved_html/www.vacourts.gov\\forms\\home.html.html"}                

    # had to open and re-save html files to utf-8
    ,{"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"WY","url":"file://../data/state_forms/_saved_html/www.courts.state.wy.us\\legal-assistance-and-forms\\court-self-help-forms.html"}
    
]

## Loop through search Pages

for i in range(0, 4):
    # is a search page, loop through all 4 (0-3, as it's zero indexed)
    states.append({"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"NE","url":"https://supremecourt.nebraska.gov/forms?page={}".format(i)})    

for i in range(1, 3):
    # is a search page, loop through all 2 (1-2, as it's NOT zero indexed)
    states.append({"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"WI","url":"https://www.wicourts.gov/forms1/circuit/ccform.jsp?page={}&FormName=&FormNumber=&beg_date=&end_date=&StatuteCite=&Category=&Language=en+&Format=2&Submit=Search".format(i)})       

for i in range(0, 16):
    # is a search page, loop through all 16 (0-15, as it's zero indexed)    
    # pulled from iframe at https://www.courts.ca.gov/forms.htm
    states.append({"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"CA","url":"https://selfhelp.courts.ca.gov/find-forms/all?page={}".format(i)})    
    
for i in range(0, 35):
    # is a search page, loop through all 35 (0-34, as it's zero indexed)
    states.append({"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":0,"jurisdiction":"VT","url":"https://www.vermontjudiciary.org/forms-library?search_api_fulltext=&page={}".format(i)})       

    
####
####  ??
####
for i in range(0, 51):
    # is a search page, loop through all 52 (0-51, as it's zero indexed)
    # result page from https://www.sccourts.org/forms/
    states.append({"regex_form":["([^\/]*\.pdf)"],"regex_link":["\/"],"goto_depth":1,"jurisdiction":"NC","url":"https://www.nccourts.gov/documents/forms?field_form_type_target_id=All&field_language_target_id=93&page={}".format(i)})
    
## Read from HTML 

In [14]:
states = [
            #{"regex":["([^/]*\.pdf)"],"goto_depth":0,"jurisdiction":"","url":""}
            #{"regex_form":["([^\/]*\.pdf)"],"regex_link":["GetForms\.aspx"],"goto_depth":2,"jurisdiction":"MN","url":"https://www.mncourts.gov/getforms.aspx"}
            {"regex_form":["([^\/]*\.pdf)"],"regex_link":["forms\/"],"goto_depth":1,"jurisdiction":"IL","url":"https://www.illinoiscourts.gov/documents-and-forms/approved-forms/"}
    
]

states

[{'regex_form': ['([^\\/]*\\.pdf)'],
  'regex_link': ['forms\\/'],
  'goto_depth': 1,
  'jurisdiction': 'IL',
  'url': 'https://www.illinoiscourts.gov/documents-and-forms/approved-forms/'}]

In [5]:
# set up

In [6]:
#for state in states:
#    print(get_soup(state["url"]))

In [23]:
%%time

visited_df = pd.DataFrame([],columns=["link"])
for state in states:
    print("======================")
    print("          %s"%state["jurisdiction"])
    print("======================")
    forms_df = pd.concat([forms_df,get_links(state["jurisdiction"],state["url"],state["regex_form"],state["regex_link"],goto_depth=state["goto_depth"],depth=0)])
    
forms_df = forms_df[~forms_df.index.duplicated(keep='first')]
display(forms_df.head())

print("\n")

if len(forms_df)>0:
    for link in forms_df.sample(n=5)["url"].values:
        print("-",link,"\n")
else:
    print("no forms found")

          MA

Checking https://www.mass.gov/guides/court-forms-by-topic
0 possible forms
45 links to follow
trying link...

Checking https://www.mass.gov/lists/probate-and-family-court-forms-for-wills-estates-and-trusts#accounting-forms-
43 possible forms
0 links to follow
43 possible forms this run

trying link...

Checking https://www.mass.gov/lists/probate-and-family-court-forms-for-actions-involving-unmarried-parents
5 possible forms
0 links to follow
5 possible forms this run

trying link...

Checking https://www.mass.gov/lists/probate-and-family-court-forms-for-adoption-and-name-change
3 possible forms
0 links to follow
3 possible forms this run

trying link...

Checking https://www.mass.gov/lists/attorney-court-forms
4 possible forms
0 links to follow
4 possible forms this run

trying link...

Checking https://www.mass.gov/lists/juvenile-court-forms#child-requiring-assistance-(cra)-
84 possible forms
0 links to follow
84 possible forms this run

trying link...

Checking https://

13 possible forms
0 links to follow
13 possible forms this run

trying link...

Checking https://www.illinoiscourts.gov/documents-and-forms/approved-forms/
14 possible forms
0 links to follow
14 possible forms this run

trying link...

Checking https://www.illinoiscourts.gov/documents-and-forms/uniform-mental-health-orders/
15 possible forms
0 links to follow
15 possible forms this run

trying link...

Checking https://www.illinoiscourts.gov/documents-and-forms/forms-probation-reimbursement-invoice-voucher/
11 possible forms
0 links to follow
11 possible forms this run

trying link...

Checking https://www.illinoiscourts.gov/documents-and-forms/annual-certification-of-private-insurance-coverage-form/
11 possible forms
0 links to follow
11 possible forms this run

trying link...

Checking https://www.illinoiscourts.gov/documents-and-forms/employment-application/
13 possible forms
0 links to follow
13 possible forms this run

trying link...

Checking https://www.illinoiscourts.gov/docume


Checking https://www.illinoiscourts.gov/documents-and-forms/approved-forms/circuit-forms/identity-theft/-declaración-jurada-de-robo-de-identidad
Error Reading Page!!!
Error reading https://www.illinoiscourts.gov/documents-and-forms/approved-forms/circuit-forms/identity-theft/-declaración-jurada-de-robo-de-identidad
trying link...

Checking https://www.illinoiscourts.gov/forms/approved-forms/forms-approved-forms-circuit-court/interpreter-request
17 possible forms
0 links to follow
17 possible forms this run

trying link...

Checking https://www.illinoiscourts.gov/forms/approved-forms/forms-approved-forms-circuit-court/limited-scope
13 possible forms
0 links to follow
13 possible forms this run

trying link...

Checking https://www.illinoiscourts.gov/forms/approved-forms/forms-approved-forms-circuit-court/mortgage-foreclosure
27 possible forms
0 links to follow
27 possible forms this run

trying link...

Checking https://www.illinoiscourts.gov/forms/approved-forms/forms-approved-forms-c

1 possible forms
0 links to follow
1 possible forms this run

trying link...

Checking https://www.mncourts.gov/GetForms.aspx?c=8
0 possible forms
0 links to follow
0 possible forms this run

trying link...

Checking https://www.mncourts.gov/GetForms.aspx?c=10
1 possible forms
0 links to follow
1 possible forms this run

trying link...

Checking https://www.mncourts.gov/GetForms.aspx?c=11
0 possible forms
0 links to follow
0 possible forms this run

trying link...

Checking https://www.mncourts.gov/GetForms.aspx?c=12
0 possible forms
0 links to follow
0 possible forms this run

trying link...

Checking https://www.mncourts.gov/GetForms.aspx?c=13
2 possible forms
0 links to follow
2 possible forms this run

trying link...

Checking https://www.mncourts.gov/GetForms.aspx?c=14
1 possible forms
0 links to follow
1 possible forms this run

trying link...

Checking https://www.mncourts.gov/GetForms.aspx?c=15
1 possible forms
0 links to follow
1 possible forms this run

trying link...

Checki

11 possible forms
0 links to follow
11 possible forms this run

trying link...

Checking https://www.courts.wa.gov/forms/?fa=forms.contribute&formID=52
10 possible forms
0 links to follow
10 possible forms this run

trying link...

Checking https://www.courts.wa.gov/forms/?fa=forms.contribute&formID=1
9 possible forms
0 links to follow
9 possible forms this run

trying link...

Checking https://www.courts.wa.gov/forms/?fa=forms.contribute&formID=73
4 possible forms
0 links to follow
4 possible forms this run

trying link...

Checking https://www.courts.wa.gov/forms/?fa=forms.contribute&formID=82
0 possible forms
0 links to follow
0 possible forms this run

trying link...

Checking https://www.courts.wa.gov/forms/?fa=forms.contribute&formID=9
6 possible forms
0 links to follow
6 possible forms this run

trying link...

Checking https://www.courts.wa.gov/forms/?fa=forms.contribute&formID=24
4 possible forms
0 links to follow
4 possible forms this run

trying link...

Checking https://www

...


Checking https://www.nccourts.gov/documents/forms?field_form_type_target_id=All&field_language_target_id=93&page=48/?field_form_type_target_id=All&field_language_target_id=93&contains=&field_form_number=&field_general_statute=&page=51
0 possible forms
0 links to follow
0 possible forms this run

37 possible forms this run

          NC

Checking https://www.nccourts.gov/documents/forms?field_form_type_target_id=All&field_language_target_id=93&page=49
0 possible forms
103 links to follow
trying link...

Checking https://www.nccourts.gov/documents/forms/request-for-transportation-order-and-order-committed-substance-abuser-fails-to-comply-with-treatment-or-is-discharged-from-24-hour-facility
1 possible forms
0 links to follow
1 possible forms this run

trying link...

Checking https://www.nccourts.gov/documents/forms/request-for-transportation-order-and-order-outpatient-fails-to-appear-for-prehearing-examination
1 possible forms
0 links to follow
1 possible forms this run

trying link...


Checking https://www.nccourts.gov/documents/forms/spc-confidential
1 possible forms
0 links to follow
1 possible forms this run

trying link...

Checking https://www.nccourts.gov/documents/forms/responsibilities-of-guardians-in-north-carolina
3 possible forms
0 links to follow
3 possible forms this run

trying link...

Checking https://www.nccourts.gov/documents/forms/order-on-motion-for-appointment-of-interim-guardian
3 possible forms
0 links to follow
3 possible forms this run

trying link...

Checking https://www.nccourts.gov/documents/forms/request-and-authorization-to-deliver-respondent
1 possible forms
0 links to follow
1 possible forms this run

trying link...

Checking https://www.nccourts.gov/documents/forms/outpatient-commitment-order-of-assignment-or-denial-of-counsel
1 possible forms
0 links to follow
1 possible forms this run

trying link...

Checking https://www.nccourts.gov/documents/forms/petition-for-authorization-to-sell-motor-vehicle-under-a-lien
1 possible forms
0 

1 possible forms
0 links to follow
1 possible forms this run

trying link...

Checking https://www.nccourts.gov/documents/forms/omb-0970-0154-instructions-income-withholding-for-support-instructions-and-omb-0970-0154-sample-form
2 possible forms
0 links to follow
2 possible forms this run

trying link...

Checking https://www.nccourts.gov/documents/forms?field_form_type_target_id=All&field_language_target_id=93&page=51/?field_form_type_target_id=All&field_language_target_id=93&contains=&field_form_number=&field_general_statute=&page=50
0 possible forms
0 links to follow
0 possible forms this run

trying link...

Checking https://www.nccourts.gov/documents/forms?field_form_type_target_id=All&field_language_target_id=93&page=51/?field_form_type_target_id=All&field_language_target_id=93&contains=&field_form_number=&field_general_statute=&page=43
0 possible forms
0 links to follow
0 possible forms this run

trying link...

Checking https://www.nccourts.gov/documents/forms?field_form_type_t

0 possible forms
0 links to follow
0 possible forms this run

0 possible forms this run

          NC

Checking https://www.nccourts.gov/documents/forms?field_form_type_target_id=All&field_language_target_id=93&page=54
0 possible forms
83 links to follow
trying link...

Checking https://www.nccourts.gov/documents/forms?field_form_type_target_id=All&field_language_target_id=93&page=54/?field_form_type_target_id=All&field_language_target_id=93&contains=&field_form_number=&field_general_statute=&page=50
0 possible forms
0 links to follow
0 possible forms this run

trying link...

Checking https://www.nccourts.gov/documents/forms?field_form_type_target_id=All&field_language_target_id=93&page=54/?field_form_type_target_id=All&field_language_target_id=93&contains=&field_form_number=&field_general_statute=&page=43
0 possible forms
0 links to follow
0 possible forms this run

trying link...

Checking https://www.nccourts.gov/documents/forms?field_form_type_target_id=All&field_language_target_i

KeyboardInterrupt: 

In [251]:
#with open(re.findall("file:\/\/(.*)","file://../data/state_forms/_saved_html/delaware_1.html")[0], 'r') as file:
#    page = file.read()
#page

In [25]:
grouped = forms_df.groupby(["jurisdiction"])[["url"]].count().sort_values(by=['url'], ascending=False)
print("Jurisdictions:",len(grouped),"\nForms:",len(forms_df))
grouped

Jurisdictions: 45 
Forms: 30901


,url
jurisdiction,
CT,5159
NC,3473
UT,2091
WA,2086
NE,1655
CO,1592
CA,1526
MA,1509
VT,1369


In [66]:
forms_df.to_csv(csv_path+"form_data.csv", index=True, encoding="utf-8")

## Download Forms

In [236]:
#for index,form in forms_df[forms_df["downloaded"]==0][:3].iterrows():
for index,form in forms_df[forms_df["downloaded"]==0].sample(n=10).iterrows():
#for index,form in forms_df[forms_df["downloaded"]==0].iterrows():
    print("Downloading: "+form["url"])
    status = download_pdf(form["url"],csv_path+form["jurisdiction"]+"/",index+".pdf")
    forms_df.xs(index)["downloaded"] = 1
    time.sleep(random.uniform(min_wait, max_wait))

Downloading: https://georgiacourts.gov//wp-content/uploads/2019/08/Jail-Instructions-Dekalb-compressed.pdf
Downloading: https://georgiacourts.gov//wp-content/uploads/2019/08/Waiver-of-Counsel-Form-Fulton.pdf
Downloading: https://www.arcourts.gov//sites/default/files/formatted-files/2017_CR_information_0.pdf
Downloading: https://courtselfhelp.idaho.gov//docs/forms/CAO_NCM_1-4.pdf
Downloading: https://www.dccourts.gov/sites/default/files/2017-06/dcca_form_appearance.pdf
Downloading: https://supremecourt.nebraska.gov/sites/default/files/JC-14-9.pdf
Downloading: https://www.courts.state.wy.us/wp-content/uploads/2017/04/STOrderDenyingMotiontoDismiss.pdf
Downloading: https://www.jud.ct.gov/webforms/forms/cl073.pdf
Downloading: https://www.courts.state.wy.us/wp-content/uploads/2017/04/DNCP17.pdf
Downloading: https://www.courts.state.wy.us/wp-content/uploads/2017/04/ECVSP02.pdf


In [237]:
print("Downloaded:",len(forms_df[forms_df["downloaded"]==1]))
print("Pending:",len(forms_df[forms_df["downloaded"]==0]))
print("All Entries:",len(forms_df))
forms_df#.sample(n=5)

Downloaded: 10
Pending: 1467
All Entries: 1477


,jurisdiction,source,group,title,url,filename,downloaded
df54030d6ab5d6ad795a9703d369a201,ME,www.courts.maine.gov,NaN,M.R. Civ. P. 45,https://www.courts.maine.gov/rules/text/MRCivP...,RULE%2045r.pdf,0
91535db09f56609ec971e09591b951ad,ME,www.courts.maine.gov,NaN,"M.R.U. Crim. P. 17, and M.R.U. Crim. P. 17A",https://www.courts.maine.gov/rules/text/mru_cr...,mru_crim_p_plus_2017-5-3.pdf,0
92956a23dc1b74344ba84e7e99e156cb,ME,www.courts.maine.gov,NaN,ADA: Accommodation Request Form,https://www.courts.maine.gov/forms/pdf/misc/ac...,accommodation-request.pdf,0
27b2c49544113c6156af1588c48ff115,ME,www.courts.maine.gov,NaN,ADA: Grievance Form for Disability Request,https://www.courts.maine.gov/forms/pdf/misc/di...,disability-accommodation-grievance.pdf,0
6c46a6b19afbee0814a92977307f9310,ME,www.courts.maine.gov,NaN,FDP Complaint Form,https://www.courts.maine.gov/forms/pdf/misc/fd...,fdp-complaint-form.pdf,0
...,...,...,...,...,...,...,...
03eef21fdd944cf30bfaeb79a108ad04,WV,www.courtswv.gov,NaN,Civil Case Information Statement,http://www.courtswv.gov/legal-community/court-...,SCA100.pdf,0
9bb3d8e7b16d91319217c0149a53cbda,WV,www.courtswv.gov,NaN,Civil Case Subpoena,http://www.courtswv.gov/legal-community/court-...,subpoena.pdf,0
ce37a3fdd727519b0eabc30545286397,WV,www.courtswv.gov,NaN,Administrative Subpoena,http://www.courtswv.gov/legal-community/court-...,admin-subpoena.pdf,0
51244eaa5b584cc012c258da4f4ecc80,WV,www.courtswv.gov,NaN,Mental Hygiene Court Contacts - Hospital Resource,http://www.courtswv.gov/legal-community/../pub...,MHContactList1-23.pdf,0


In [275]:
forms_df.to_csv(csv_path+"form_data.csv", index=True, encoding="utf-8")